# check GPU availabilty

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Number of GPUs Available: ", len(physical_devices))
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

Number of GPUs Available:  0


In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
DRIVE_PATH = '/content/drive/MyDrive/Bachelor_Project/'
#%cd '/content/drive/MyDrive/Bachelor_Project/'
#%ls

Mounted at /content/drive/


# Load Dataset from kaggle

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#! mkdir ~/.kaggle
#! cp kaggle.json ~/.kaggle/
#! chmod 600 ~/.kaggle/kaggle.json
#! kaggle datasets download mozillaorg/common-voice

In [ ]:
#!unzip /content/common-voice.zip

# spoken-language-identification dataset


In [ ]:
! mkdir ~/.kaggle
! cp "/content/drive/MyDrive/Bachelor_Project/kaggle.json" ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download toponowicz/spoken-language-identification

100% 14.9G/14.9G [03:30<00:00, 78.9MB/s]
100% 14.9G/14.9G [03:30<00:00, 76.2MB/s]


In [ ]:
!unzip /content/spoken-language-identification.zip
!mkdir /content/train/train-img/
!mkdir /content/test/test-img/

Streaming output truncated to the last 5000 lines.
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.pitch5.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.pitch6.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.pitch7.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.pitch8.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.speed1.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.speed2.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.speed3.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.speed4.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.speed5.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragment16.speed6.flac  
  inflating: train/train/es_m_b53274d3ef7996a22d6ec929c1869804.fragme

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import librosa
import librosa.display
import gc

In [ ]:
Data_dir=np.array(glob("/content/train/train/*"))

In [ ]:
print(len(Data_dir))

73080


## Create spectogram

In [ ]:
train_size= 100
test_size= 100

In [ ]:
def create_spectrogram(filename,name, is_train=True):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    #S = librosa.feature.mfcc(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    prefix= '/content/train/train-img/'
    if not is_train:
        prefix= '/content/test/test-img/'
    filename  = prefix + name
    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

In [ ]:
%%time
i=0
train_id = []
for file in Data_dir[i:i+train_size]:
    name = file.split('/')[-1] + '.jpg'
    train_id.append(name+".jpg")
    create_spectrogram(file,name)

traindf = pd.DataFrame(Data_dir[i:i+train_size],columns = ['ID'])

CPU times: user 32.8 s, sys: 11.3 s, total: 44.1 s
Wall time: 36.9 s


In [ ]:
%%time
#for file in Data_dir[i:i+train_size]:
#    name = file.split('/')[-1]
#    train_id.append(name+".jpg")
#    create_spectrogram(file,name)

traindf = pd.DataFrame(Data_dir[0:train_size],columns = ['file'])
traindf['ID']= traindf['file'].apply(lambda x: x.split('/')[-1]+'.jpg')
#traindf.apply(lambda x: create_spectrogram(x.file, x.ID), axis=1)
np.vectorize(create_spectrogram)(traindf['file'], traindf['ID'])
traindf

CPU times: user 22.7 s, sys: 11.1 s, total: 33.8 s
Wall time: 22.6 s


,file,ID
0,/content/train/train/en_f_386ee651f6f1539ff562...,en_f_386ee651f6f1539ff5622c55e234e5a4.fragment...
1,/content/train/train/es_f_53b555eab2b3baada380...,es_f_53b555eab2b3baada380f7d3ede20b20.fragment...
2,/content/train/train/es_m_534a0cf6f6aea8e35667...,es_m_534a0cf6f6aea8e35667a9147d2d0f8d.fragment...
3,/content/train/train/de_m_df90bffac026f705f4a9...,de_m_df90bffac026f705f4a99871d71684f1.fragment...
4,/content/train/train/en_m_61764486dd621ce03fa4...,en_m_61764486dd621ce03fa461e92cd9e2b3.fragment...
...,...,...
95,/content/train/train/en_f_10134f409d9b7b0b95fe...,en_f_10134f409d9b7b0b95fed6e025febcad.fragment...
96,/content/train/train/de_m_df90bffac026f705f4a9...,de_m_df90bffac026f705f4a99871d71684f1.fragment...
97,/content/train/train/es_m_d5b91a4ffb1ead826b79...,es_m_d5b91a4ffb1ead826b7968ec19cbfa1c.fragment...
98,/content/train/train/es_f_bac9be13a058d1c859ed...,es_f_bac9be13a058d1c859ed5d2e3a0dc27b.fragment...


In [ ]:
traindf['lang'] = traindf['ID'].apply(lambda x: x.split('_')[0])

# save data to drive

In [ ]:
traindf.to_csv(DRIVE_PATH+'traindf.csv')

In [ ]:
#!cp traindf.csv $DRIVE_PATH
!cp /content/train/train-img/ $DRIVE_PATH -r
#!echo $DRIVE_PATH

# read data from drive

In [ ]:
traindf=pd.read_csv(DRIVE_PATH+'traindf.csv')

In [ ]:
#import shutil
#shutil.rmtree('/content/train/train-img/', ignore_errors=True)

In [ ]:
!cp $DRIVE_PATH/train-img/ /content/train/train-img/ -r

In [ ]:
!rm /content/spoken-language-identification.zip

# Parallelism

In [ ]:
!pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 830.9/830.9 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.9 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16299 sha256=991106cbdbafc63cc368ef13a5c47bf0ed73ae6e3b0db6dfe3d8d06fe82d9987
  Stored in directory: /root/.cache/pip/wheels/6c/bd/3e/2d6afc9bc36c9975f8e4215a270bbac6580c4361ebd6bb2323
Successfully built swifter


In [ ]:
import swifter
traindf.swifter.apply(lambda x: create_spectrogram(x.file, x.ID), axis=1)

In [ ]:
    #! rm /content/train/*.jpg

In [ ]:
gc.collect()

27394

In [ ]:
#print(train_id)
#train_lang = []
#for i in range(len(train_id)):
#    train_lang.append(train_id[i].split('_')[0])
#traindf=pd.DataFrame(train_id,columns = ['ID'])
#
#traindf['lang'] = traindf['ID'].apply(lambda x: x.split('_')[0])

In [ ]:
#traindf['lang'] = train_lang
#traindf.drop(columns='Class', inplace=True)

In [ ]:
traindf

,file,ID,lang
0,/content/train/train/de_f_8e8ed685b0cf9f5bbd13...,de_f_8e8ed685b0cf9f5bbd13a04629ee64c5.fragment...,de
1,/content/train/train/en_m_6876763e37d7a41d6f53...,en_m_6876763e37d7a41d6f53f4322a72b0d2.fragment...,en
2,/content/train/train/es_m_b8e0e6f56f02e6f8f79c...,es_m_b8e0e6f56f02e6f8f79cc360958e5982.fragment...,es
3,/content/train/train/es_m_7e1c8fd8bdbbd6844ddd...,es_m_7e1c8fd8bdbbd6844ddddbdf889f3364.fragment...,es
4,/content/train/train/de_f_9bd3b58fe6f75dc40e85...,de_f_9bd3b58fe6f75dc40e85021914b4b06a.fragment...,de
...,...,...,...
9995,/content/train/train/en_m_81995ee8a5e990193b78...,en_m_81995ee8a5e990193b7858ec4b158e48.fragment...,en
9996,/content/train/train/de_f_8e8ed685b0cf9f5bbd13...,de_f_8e8ed685b0cf9f5bbd13a04629ee64c5.fragment...,de
9997,/content/train/train/en_m_d630900a88e2681c9d5d...,en_m_d630900a88e2681c9d5d1026d7025d67.fragment...,en
9998,/content/train/train/es_f_56ef0bde6c92a0100da2...,es_f_56ef0bde6c92a0100da23cdd39a8a112.fragment...,es


In [ ]:
%%time
i= train_size #start i after train_size
test_id = []
for file in Data_dir[i:i+test_size]:
    name = file.split('/')[-1]
    test_id.append(name+'.jpg')
    create_spectrogram(file,name, is_train=False)

CPU times: user 18.6 s, sys: 10.2 s, total: 28.8 s
Wall time: 18.7 s


In [ ]:
testdf=pd.DataFrame(test_id,columns = ['ID'])

test_lang = []
for i in range(len(test_id)):
    test_lang.append(test_id[i].split('_')[0])

testdf['lang'] = test_lang


In [ ]:
testdf

,ID,lang
0,es_f_eca5ab833fd0b8e6d9b77a7df5d8610d.fragment...,es
1,es_m_b8e0e6f56f02e6f8f79cc360958e5982.fragment...,es
2,es_f_68e1dcf1b41edcfa24b402f46ec2a65e.fragment...,es
3,es_f_68e1dcf1b41edcfa24b402f46ec2a65e.fragment...,es
4,en_m_6876763e37d7a41d6f53f4322a72b0d2.fragment...,en
...,...,...
95,de_f_26ddc4e756f61233d74d42cc000c9dae.fragment...,de
96,de_f_d94712992f41e3d8d21f22274b3d8fd9.fragment...,de
97,en_m_81995ee8a5e990193b7858ec4b158e48.fragment...,en
98,de_m_3aa01c55ba7ca85cad2977c848eb826f.fragment...,de


In [ ]:
!pip install Keras-Preprocessing
from keras_preprocessing.image import ImageDataGenerator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.2).flow_from_


train_generator=datagen.flow_from_dataframe(
    dataframe=traindf,
    directory="/content/train/train-img",
    x_col="ID",
    y_col="lang",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64,64))

valid_generator=datagen.flow_from_dataframe(
    dataframe=traindf,
    directory="/content/train/train-img",
    x_col="ID",
    y_col="lang",
    subset="validation",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64,64))



Found 8000 validated image filenames belonging to 3 classes.
Found 2000 validated image filenames belonging to 3 classes.


In [ ]:
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import keras

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0005, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 64, 64, 32)        896       
                                                                 
 activation_14 (Activation)  (None, 64, 64, 32)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 62, 62, 64)        18496     
                                                                 
 activation_15 (Activation)  (None, 62, 62, 64)        0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 31, 31, 64)       0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 31, 31, 64)        0         
                                                      

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=150
)
model.evaluate_generator(generator=valid_generator, steps=STEP_SIZE_VALID
)

<ipython-input-71-f86d2eeb155e>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator,


Epoch 1/150
250/250 [==============================] - 35s 133ms/step - loss: 1.0952 - accuracy: 0.3654 - val_loss: 1.0755 - val_accuracy: 0.3810
Epoch 2/150
250/250 [==============================] - 15s 60ms/step - loss: 1.0462 - accuracy: 0.4319 - val_loss: 1.0030 - val_accuracy: 0.4743
Epoch 3/150
250/250 [==============================] - 15s 60ms/step - loss: 0.9817 - accuracy: 0.5010 - val_loss: 1.0515 - val_accuracy: 0.4390
Epoch 4/150
250/250 [==============================] - 15s 60ms/step - loss: 0.8966 - accuracy: 0.5723 - val_loss: 0.8384 - val_accuracy: 0.6260
Epoch 5/150
250/250 [==============================] - 18s 72ms/step - loss: 0.8251 - accuracy: 0.6255 - val_loss: 0.7168 - val_accuracy: 0.6885
Epoch 6/150
250/250 [==============================] - 15s 61ms/step - loss: 0.7519 - accuracy: 0.6646 - val_loss: 0.7335 - val_accuracy: 0.7177
Epoch 7/150
250/250 [==============================] - 15s 60ms/step - loss: 0.6929 - accuracy: 0.6946 - val_loss: 0.6290 - val_a

<ipython-input-71-f86d2eeb155e>:9: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(generator=valid_generator, steps=STEP_SIZE_VALID


[0.3116071820259094, 0.8946572542190552]

In [ ]:
model.save('/content/model/Model_CNN1000.h5')
gc.collect()

616

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
    dataframe=testdf,
    directory="/content/test/test-img/",
    x_col="ID",
    y_col='lang',
    batch_size=32,
    seed=42,
    shuffle=False,
    class_mode='categorical',
    target_size=(64,64))
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

Found 100 validated image filenames.


In [ ]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)

<ipython-input-74-f904d4e6328a>:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred=model.predict_generator(test_generator,


3/3 [==============================] - 0s 39ms/step


In [ ]:
predicted_class_indices.shape

(96,)

In [ ]:
from PIL import Image
import numpy as np
from skimage import transform
def load(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (64, 64, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

image = load('/content/test/test-img/de_f_0809fd0642232f8c85b0b3d545dc2b5a.fragment18.pitch2.flac.jpg')
model.predict(image)

1/1 [==============================] - 0s 21ms/step


array([[9.9764061e-01, 1.9771063e-03, 3.8233039e-04]], dtype=float32)

In [ ]:
train_generator.class_indices

{'de': 0, 'en': 1, 'es': 2}

In [ ]:
prediction = model.predict(x=)